In [2]:
!pip install accelerate peft pandas sentence-transformers sentencepiece nltk
!pip install -i https://pypi.org/simple/ bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.2 MB/s eta 0:00:0000:0100:010m
Looking in indexes: https://pypi.org/simple/


In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — русскоязычный ассистент специалиста подбора персонала. Отвечаешь исключительно на русском языке. Ты помогаешь анализировать вакансии, извлекать из вакансий список требований и навыков. Ты не используешь информацию которой нет в тексте вакансий."
TEXT2SKILLS_MESSAGE = "Составь нумерованный список технологий которыми требуется владеть для данной вакансии. Текст вакансии: "

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=BitsAndBytesConfig(load_in_4bit=True),
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.bfloat16
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.repetition_penalty = 1.1
generation_config.temperature = 0.02
generation_config.max_new_tokens = 375
generation_config.top_p = 0.90
generation_config.top_k = 40

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 375,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.02,
  "top_k": 40,
  "top_p": 0.9
}



In [9]:
# U['text'] = U['descs'] + ' ' + U['key_skills'] + ' ' + U['roles']

In [4]:
def vacancy_to_skills(extended_description):
    conversation = Conversation()
    conversation.add_user_message(TEXT2SKILLS_MESSAGE + re.sub(r'<.*?>', '', extended_description))
    prompt = conversation.get_prompt(tokenizer)
    return generate(model, tokenizer, prompt, generation_config)

In [8]:
print(vacancy_to_skills('Сетевой инженер Мы - Wildberries, крупнейший маркетплейс России. Wildberries - это 30 млн пользователей ежедневно, сотни тысяч сотрудников и одна из сильнейших IT-команд в стране. Наша компания стабильно развивается и работает в усиленном режиме: гибкий современный стек и около-rocket-science задачи прилагаются.Что нужно будет делать:Построение и обслуживание сетей дата центров;Участие в проектах и решение вопросов смежных команд в качестве сетевого эксперта;Описание технических Work Flow в команде и выработка типовых решений;Решение инцидентов во время дежурств, возникающих на сети передачи данных;Что хотели бы видеть:Глубокие знания сетевых технологий;Протоколов динамической маршрутизации (OSPF, BGP, ISIS);MPLS/EVPN/DCI;Модели OSI, стека TCP/IP.Понимание принципов построения сетей современных центров обработки данных;Понимание принципов OpenFlow/SDN;Принципов балансировки трафика;Опыт работы с *nix системами;Владение инструментами сетевой диагностики (системы мониторинга, средства анализа и отладки);Навык диагностики сетевых проблем не по инструкции;Представление о переподписке на фабриках центров обработки данных;Мы предлагаем:График 5/2 с 9:00 до 18:00 МСК; Возможны несколько вариантов на выбор: полная занятость в офисе, полностью удаленная или гибридная (офис+удаленка);Сложные и интересные проекты;Возможности для профессионального развития и роста;Участие в конференциях, Meetups и Tech Talks;Позитивная атмосфера и дружный коллектив; Возможен корпоративный ДМС;'))

1. Глубокие знания сетевых технологий
2. Протоколы динамической маршрутизации (OSPF, БГП, ИСИС)
3. MPLS/EVPN/DCI
4. Модели ОСИ, стека TCP/IP
5. Понимание принципов построения сетей современных ЦОД
6. Понимание принципов OpenFlow/SDN
7. Принципы балансировки трафика
8. Опыт работы с *nix системами
9. Владение инструментами сетевой диагностики
10. Навык диагностики сетевых проблем непо инструкции
11. Представление о переподписке на фабриках ЦОД
12. График 5/2 с 9:00 до18:00 МСК
13. Возможные варианты на выбор: полная занятость в офисе, удаленная или гибридная (офис+удоленка)
14. Сложные и интересные проекты
15. Возможности для профессионального развития и роста
16. Участие в конференциях, Meetups и Тех Talks
17. Позитивная атмосфера и дружный колектив
18. Возможен корпоративный ДМС
